In [1]:
from ggmodel_dev.models.water.EW import model_properties
from ggmodel_dev.models.utils import all_model_dictionary, all_model_properties_df, all_model_properties


In [ ]:
import os
import pandas as pd
import plotly.express as px

from plots.simulation.GE3_plots import sankeyplot, emission_data_dict_to_df
from ggmodel_dev.models.landuse import BE2_scenario, GE3_scenario
from ggmodel_dev.models.water import EW_scenario
from ggmodel_dev.models.transport import VEHC_scenario
from ggmodel_dev.projection import run_projection



def get_data_dict_from_folder_parquet(folder_name):
    files = os.listdir(folder_name)
    data_dict = {file.split('.')[0]: pd.read_parquet(
        f'{folder_name}/{file}') for file in files}
    data_dict = {name: df[name] for name, df in data_dict.items()}
    return data_dict


def get_data_dict_from_folder(folder_name):
    files = os.listdir(folder_name)
    data_dict = {file.split('.')[0]: pd.read_csv(
        f'{folder_name}/{file}') for file in files}

    data_dict = {name: df.set_index([col for col in df.columns if col != name]).squeeze(
    ) for name, df in data_dict.items()}

    return data_dict

In [ ]:
scenario_data_dictionnary = {
    'EW_models': get_data_dict_from_folder('data/sim/EW'),
    'BE2_model': get_data_dict_from_folder('data/sim/BE2'),
    'GE3_model': get_data_dict_from_folder_parquet('data/sim/GE3'),
    'VEHC_model': get_data_dict_from_folder('data/sim/VEHC'),
}


In [ ]:
from ggmodel_dev.projection import *
from ggmodel_dev.models.landuse.GE3 import model_dictionnary

projection_dict = {
    'Pop': lambda x: x,
    'OEi': lambda x: x,
    'EF_EEi': lambda x: x,
    'TAi': lambda x: x,
    'IN_F': lambda x: x,
    'MYi': lambda x: x,
    'EF_ASi': lambda x: x,
    'EF_Ti': lambda x: x,
    'EF_CH4Ti': lambda x: x,
    'EF_Li': lambda x: x,
    'EF_F': lambda x: x,        
}


def run_scenario(data_dict,
                 MM_Ti=1/3,
                 MM_ASi=1/3,
                 MM_LPi=1/3,
                 TAi_pct_Asses=1,
                 TAi_pct_Cattle_dairy=1,
                 TAi_pct_Cattle_non_dairy=1,
                 TAi_pct_Chickens_broilers=1,
                 TAi_pct_Chickens_layers=1,
                 TAi_pct_Ducks=1,
                 TAi_pct_Goats=1,
                 TAi_pct_Horses=1,
                 TAi_pct_Mules=1,
                 TAi_pct_Sheep=1,
                 TAi_pct_Swine_breeding=1,
                 TAi_pct_Swine_market=1,
                 TAi_pct_Turkeys=1
                 ):
    
    data_dict = data_dict.copy()
    
    
    data_dict['MM_Ti'] = MM_Ti
    data_dict['MM_ASi'] = MM_ASi
    data_dict['MM_LPi'] = MM_LPi
    
    TAi_pct_target = pd.Series(index=['Asses', 'Cattle, dairy','Cattle, non-dairy',
                                  'Chickens, broilers', 'Chickens, layers', 'Ducks', 'Goats',
                                  'Horses', 'Mules', 'Sheep', 'Swine, breeding', 'Swine, market',
                                  'Turkeys'],
                           data=[TAi_pct_Asses, TAi_pct_Cattle_dairy,
                                 TAi_pct_Cattle_non_dairy, TAi_pct_Chickens_broilers,
                                 TAi_pct_Chickens_layers, TAi_pct_Ducks,TAi_pct_Goats,
                                 TAi_pct_Horses, TAi_pct_Mules ,TAi_pct_Sheep, TAi_pct_Swine_breeding,
                                 TAi_pct_Swine_market, TAi_pct_Turkeys
                               
                           ], name='TAi_pct_target')
    
    TAi_pct_target.index.name = 'Item'
    
    tmp = pd.merge(data_dict['TAi'].reset_index(), TAi_pct_target.reset_index(),on='Item')
    tmp['TAi'] = tmp['TAi'] * tmp['TAi_pct_target']

    data_dict['TAi'] = tmp.set_index(['ISO', 'Year', 'Item'])['TAi']
    
    results = model_dictionnary['GE3_model'].run(data_dict)
    
    return results

In [ ]:
def emission_data_dict_to_df(data_dict):

    data_dict = data_dict.copy()

    data = pd.concat([v.to_frame(name='Value').assign(Variable=k)
                      for k, v in data_dict.items()], axis=0).reset_index().dropna()

    data = pd.concat([data, data.groupby('Variable').sum().reset_index().rename(
        columns={"Variable": 'Item'}).assign(Variable='GE3_partial')])

    return data

In [ ]:
from ggmodel_dev.models.landuse import BE2_scenario, GE3_scenario, GE3

def run_all_scenarios_GE3(data_dict, ISO, args_dict_1, args_dict_2):
    scenarios_results = {}
    data_dict = {k: v.loc[ISO, 2018, :] for k, v in data_dict.items()}

    scenarios_results['BAU'] = GE3_scenario.run_scenario(
        data_dict=data_dict, MM_Ti=data_dict['MM_Ti'], MM_ASi=data_dict['MM_ASi'])
    scenarios_results['scenario_one'] = GE3_scenario.run_scenario(
        data_dict=data_dict, **args_dict_1)
    scenarios_results['scenario_two'] = GE3_scenario.run_scenario(
        data_dict=data_dict, **args_dict_2)
    
    displayed_variables = ['TEE_CO2eq', 'TMA_CO2eq', 'TMT_CO2eq', 'TMP_CO2eq']

    df_0 = emission_data_dict_to_df({k: v for k, v in scenarios_results['BAU'].items() if k in displayed_variables})
    df_1 = emission_data_dict_to_df({k: v for k, v in scenarios_results['scenario_one'].items() if k in displayed_variables})
    df_2 = emission_data_dict_to_df({k: v for k, v in scenarios_results['scenario_two'].items() if k in displayed_variables})

    df = pd.concat([df_0.assign(scenario='BAU'), df_1.assign(scenario='scenario 1'), df_2.assign(scenario='scenario 2')], axis=0)

    df = df.merge(GE3.model_dictionnary['GE3_model'].summary_df[['name', 'unit']], left_on='Variable', right_index=True, how='left')

    #fig_2 = px.treemap(df.query('Variable != "GE3_partial"'), path=['Item', 'name', 'scenario'], values='Value', )
    fig_1 = px.treemap(df.query('Variable != "GE3_partial"'), path=['scenario', 'Item', 'name'], values='Value',  color='scenario', color_discrete_map={'BAU': 'grey', 'scenario 1': '#D8A488', 'scenario 2': '#86BBD8'})

    
    df.loc[df.Variable == 'GE3_partial', 'unit'] = 'gigagrams (CO2eq)'
    df.loc[df.Variable == 'GE3_partial', 'name'] = 'Non-CO2 agricultural emissions'


    df = df.merge(model_dictionnary['GE3_model'].summary_df[['name']], left_on='Item', right_index=True, how='left', suffixes=('_target', '_source'))
    df['name'] = df['name_target']
    df.loc[df.name_source.isna(), 'name'] = df.loc[df.name_source.isna(), 'Item']

    df.loc[df.name_source.isna(), 'name_source'] = df.loc[df.name_source.isna(), 'Item']

    df.loc[df.name_target.isna(), 'name_target'] = df.loc[df.name_target.isna(), 'Variable']
    
    fig_2 = sankeyplot(df, 'name_source', 'name_target')
    return fig_1, fig_2, {}


In [ ]:
data_dict = scenario_data_dictionnary['GE3_model']

args_dict_1 = dict(MM_Ti=1/3, MM_ASi=1/3, MM_LPi=1/3)
args_dict_2 = dict(MM_Ti=1/3, MM_ASi=1/3, MM_LPi=1/3, TAi_pct_Cattle_non_dairy=1/2)

fig_1, fig_2, __ = run_all_scenarios_GE3(data_dict, 'FRA', args_dict_1, args_dict_2)
 

#sankeyplot(df, 'Item', 'Variable')

In [ ]:
fig_2

In [ ]:
from ggmodel_dev.models.landuse.GE3 import model_dictionnary
df = df.merge(model_dictionnary['GE3_model'].summary_df[['name', 'unit']], left_on='Variable', right_index=True, how='left')

df.loc[df.Variable == 'GE3_partial', 'unit'] = 'gigagrams (CO2eq)'
df.loc[df.Variable == 'GE3_partial', 'name'] = 'Non-CO2 agricultural emissions'


df = df.merge(model_dictionnary['GE3_model'].summary_df[['name']], left_on='Item', right_index=True, how='left', suffixes=('_target', '_source'))
df['name'] = df['name_target']
df.loc[df.name_source.isna(), 'name'] = df.loc[df.name_source.isna(), 'Item']

df.loc[df.name_source.isna(), 'name_source'] = df.loc[df.name_source.isna(), 'Item']

df.loc[df.name_target.isna(), 'name_target'] = df.loc[df.name_target.isna(), 'Variable']

In [ ]:
df

In [ ]:
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go
import pandas as pd


def emission_data_dict_to_df(data_dict):

    data_dict = data_dict.copy()

    data = pd.concat([v.to_frame(name='Value').assign(Variable=k)
                      for k, v in data_dict.items()], axis=0).reset_index().dropna()

    data = pd.concat([data, data.groupby('Variable').sum().reset_index().rename(
        columns={"Variable": 'Item'}).assign(Variable='Non-CO2 agricultural emissions')])

    return data


def encode_source_target(df, source='Item', target='Variable'):

    le = LabelEncoder()

    encoded = le.fit_transform(
        df[[source, target]].values.flatten()).reshape(-1, 2)

    return encoded, le.classes_


def sankeyplot(df, source, target, valueformat='.00f', valuesuffix=' Gg(CO2eq)'):

    data = df.copy()

    encoded_s_t, classe_names = encode_source_target(data, source, target)

    data[['Source', 'Target']] = encoded_s_t
    color_dict = pd.DataFrame({'scenario': ['BAU', 'scenario 1' ,'scenario 2'],
                               'color': ['grey', '#D8A488', '#86BBD8']})
    
    data = data.merge(color_dict, on='scenario')
    
    fig = go.Figure(data=[go.Sankey(
        valueformat=valueformat,
        valuesuffix=valuesuffix,
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=classe_names,
            color='lightgrey'
        ),
        link=dict(
            target=data['Target'],
            source=data['Source'],
            value=data['Value'],
            color=data['color']
        ))])

    return fig

In [ ]:
sankeyplot(df, 'name_source', 'name_target')